In [ ]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
from PIL import Image

In [ ]:

START = 0x08
RUN = 0x02
JUMP = 0x01
mario_y = 0x00CE
mario_x = 0x006D

def hold(mask, duration):
    return [mask for i in range(duration)]



imgBuffer = VectorBytes()
def outputImage(emu, name, buf=imgBuffer):
    emu.imageInto(buf)
    outImg = Image.frombytes("RGBA", (256, 256), str(bytearray(buf)))
    outImg.save(name + ".png")

In [ ]:

emu = fceulib.runGame('mario.nes')
startInputs = hold(0x0, 120) + hold(START | JUMP, 30) + hold(0x0, 150)
jumpInputs = [hold(0x0, 5) + hold(JUMP, t) + hold(0x0, 300)
              for t in range(5, 300)]
for m in startInputs:
    emu.step(m, 0x0)
emu.stepFull(0x0, 0x0)
start = VectorBytes()
print("SAVE")
emu.save(start)

In [ ]:
def pointer_to_numpy(ptr,length=0):
    if length ==0:
        length = len(ptr)
    return np.array([ptr[xx] for xx in range(length)])

emu.load(start)
nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
ram_ptr = emu.fc.fceu.RAM
nt = nt[:960].reshape(30,32)
print nt
outputImage(emu, "start")

In [ ]:
import numpy as np
nt = np.array(nt)

import matplotlib.pyplot as plt
#plt.imshow(nt,interpolation='none')
#plt.show()

In [ ]:
pt = pointer_to_numpy(emu.fc.ppu.PALRAM)
print pt

In [ ]:
sprite_ram  = pointer_to_numpy(emu.fc.ppu.SPRAM)
def sprite_attributes_to_dict(attributes):
    masks = {'background' : 0x20,
        'hflip' : 0x40,
        'vflip'  : 0x80}
    sprite_attributes = {mask:attributes&masks[mask]>0 for mask in masks}
    sprite_attributes['palette'] = attributes & 0x03
    return sprite_attributes
to_plot = []
for ii in range(0,len(sprite_ram),4):
    if sprite_ram[ii] < 240:
        print ii,'x: {}, y: {}, id:{}, attributes:{},{}'.format(sprite_ram[ii+3],
                                                      sprite_ram[ii],
                                                      sprite_ram[ii+1],
                                                      sprite_attributes_to_dict(sprite_ram[ii+2]),
                                                      sprite_ram[ii+2])
    if sprite_ram[ii+1] > 0:
        to_plot.append([sprite_ram[ii+3],sprite_ram[ii]])
to_plot = np.array(to_plot,dtype='float')
print to_plot
plt.plot(to_plot[:,0]/8.,to_plot[:,1]/8.,'rx')
plt.imshow(nt,interpolation='none')
plt.show()


In [ ]:
sprite_id = 7
def fetch_sprite(sprite_id,attributes):
    sprite = []
    for yy in range(8):
        a = emu.fc.ppu.fetchSprite(sprite_id,yy)
        row = []
        for xx in range(8):
            row.append(a&3)
            a >>=4
        if attributes['hflip']:
            sprite.append(list(reversed(row)))
        else:
            sprite.append(row)
    sprite = np.array(sprite)
    return sprite
plt.imshow(fetch_sprite(sprite_id,sprite_attributes_to_dict(sprite_ram[sprite_id*4+2])),interpolation='nearest')
plt.show()

In [ ]:
bg_pat_addr = 0x1000 if (emu.fc.ppu.values[0] & (1 << 4)) else 0x0000;
bg_ram = pointer_to_numpy(emu.fc.cart.getVPageChunk(bg_pat_addr),0x1000)
def get_tile(tile_id):
    tile = []
    for yy in range(8):
        row = []
        lb = bg_ram[tile_id*16+yy]
        ub = bg_ram[tile_id*16+yy+8]
        for xx in range(8):
            l = lb&1
            u = ub&1
            v = l+u*2
            lb >>= 1
            ub >>= 1
            row.append(v)
        tile.append(list(reversed(row)))
    tile = np.array(tile)
    return tile
plt.imshow(get_tile(181),interpolation='nearest')
plt.show()
#  const uint8 *vram = &emu->GetFC()->cart->
# VPage[bg_pat_addr >> 10][bg_pat_addr];

In [ ]:
help(fceulib)

In [ ]:

from collections import defaultdict
import zipfile
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import animation


class ROMViewer(object):
    """Visually inspect an NES ROM"""
    def __init__(self, filename, N1=16, N2=16, sep=1):
        if zipfile.is_zipfile(filename):
            zp = zipfile.ZipFile(filename)
            data = np.unpackbits(np.frombuffer(zp.read(zp.filelist[0]),
                                               dtype=np.uint8))
        else:
            data = np.unpackbits(np.fromfile(filename, dtype=np.uint8))

        self.data = data.reshape((-1, 8, 8))

In [ ]:
graphics = ROMViewer('mario.nes')
import scipy.misc
scipy.misc.imsave('outfile.png', graphics.data.reshape([-1,8]))